In [5]:
# Install required libraries
!pip install geopandas shapely pandas tqdm

import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon, LineString, box
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import math
from collections import defaultdict
import json

def load_datasets(dataset_names):
    datasets = {}
    for name in dataset_names:
        try:
            data = gpd.read_file(f"{name}.geojson")
            for col in data.columns:
                if data[col].dtype in ['float64', 'float32']:
                    data[col] = data[col].fillna(0).astype(str)
                elif data[col].dtype == 'object':
                    data[col] = data[col].fillna('Unknown')
            # Reset index to ensure no float indices
            data = data.reset_index(drop=True)
            datasets[name] = data
            print(f"Loaded {name} with {len(datasets[name])} features")
        except Exception as e:
            print(f"Error loading {name}: {e}")
    return datasets

def to_safe_geojson(gdf, properties=None):
    if properties is None:
        properties = [col for col in gdf.columns if col != 'geometry']
    gdf_filtered = gdf[properties + ['geometry']].copy()
    for col in gdf_filtered.columns:
        if col != 'geometry':
            gdf_filtered[col] = gdf_filtered[col].astype(str)
    geojson = gdf_filtered.to_json()
    data = json.loads(geojson)
    for feature in data['features']:
        feature['properties'] = {str(k): str(v) for k, v in feature['properties'].items()}
    return data

dataset_names = [
    "Street_Light_Poles_and_Luminaires", "Traffic_Collisions", "hamilton_bike_lanes_osm",
    "hamilton_pedestrian_osm", "hamilton_streets_osm", "Road_Sidewalk", "Street_Centreline",
    "Transit_Service_Areas", "HSR_Bus_Stops", "Hospitals", "EMS_Stations", "Police_Stations",
    "Educational_Institutions", "Municipal_Services_Centres", "Places_of_Worship", "Arenas",
    "Spray_Pads", "Libraries", "Museums_and_Galleries", "Public_Art_and_Monuments",
    "City_Waterfalls", "Recreation_and_Community_Centres", "Park_Sports_Fields", "Campgrounds",
    "Ward_Boundaries", "Bikeways", "Bike_Parking", "Hamilton_Bike_Share_Incorporated_Hubs",
    "Hamilton_Bike_Share_Incorporated_Service_Areas", "City_Growth_Targets_Population"
]

print("Loading datasets...")
datasets = load_datasets(dataset_names)

if 'Ward_Boundaries' not in datasets or len(datasets['Ward_Boundaries']) == 0:
    print("Error: Ward boundaries dataset is missing or empty.")
    hamilton_bbox = box(-80.2, 43.2, -79.7, 43.3)
    hamilton_boundary = gpd.GeoDataFrame({'geometry': [hamilton_bbox]}, crs="EPSG:4326")
else:
    hamilton_boundary = datasets['Ward_Boundaries']

for name, data in datasets.items():
    if data.crs is None:
        print(f"Warning: {name} has no CRS, assuming WGS 84 (EPSG:4326)")
        data.crs = "EPSG:4326"
    elif data.crs != "EPSG:4326":
        print(f"Reprojecting {name} from {data.crs} to EPSG:4326")
        data = data.to_crs("EPSG:4326")
        datasets[name] = data

def create_grid(boundary_gdf, cell_size_meters=400):
    boundary_projected = boundary_gdf.to_crs("EPSG:32617")
    minx, miny, maxx, maxy = boundary_projected.total_bounds
    cols = int(np.ceil((maxx - minx) / cell_size_meters))
    rows = int(np.ceil((maxy - miny) / cell_size_meters))
    grid_cells = [box(minx + i * cell_size_meters, miny + j * cell_size_meters,
                      minx + (i + 1) * cell_size_meters, miny + (j + 1) * cell_size_meters)
                  for i in range(cols) for j in range(rows)]
    grid = gpd.GeoDataFrame({'geometry': grid_cells}, crs="EPSG:32617").to_crs("EPSG:4326")
    grid = gpd.sjoin(grid, boundary_projected.to_crs("EPSG:4326"), how="inner", predicate="intersects")[['geometry']]
    grid['cell_id'] = [str(i) for i in range(len(grid))]
    return grid

print("Creating grid...")
grid = create_grid(hamilton_boundary)
print(f"Created {len(grid)} grid cells")

def calculate_walkability_indicators(grid, datasets):
    grid['sidewalk_length'] = 0.0
    grid['street_light_count'] = 0
    grid['pedestrian_path_length'] = 0.0
    grid['bus_stop_count'] = 0
    grid['collision_count'] = 0
    grid['amenity_count'] = 0
    grid['amenity_variety'] = 0
    grid['street_density'] = 0.0
    grid['intersection_density'] = 0.0

    if 'Road_Sidewalk' in datasets and not datasets['Road_Sidewalk'].empty:
        for idx, cell in tqdm(grid.iterrows(), total=len(grid), desc="Calculating sidewalk length"):
            intersecting = datasets['Road_Sidewalk'][datasets['Road_Sidewalk'].intersects(cell.geometry)]
            if not intersecting.empty:
                grid.at[idx, 'sidewalk_length'] = intersecting.to_crs("EPSG:32617").length.sum()

    if 'Street_Light_Poles_and_Luminaires' in datasets and not datasets['Street_Light_Poles_and_Luminaires'].empty:
        for idx, cell in tqdm(grid.iterrows(), total=len(grid), desc="Counting street lights"):
            grid.at[idx, 'street_light_count'] = datasets['Street_Light_Poles_and_Luminaires'].intersects(cell.geometry).sum()

    if 'hamilton_pedestrian_osm' in datasets and not datasets['hamilton_pedestrian_osm'].empty:
        for idx, cell in tqdm(grid.iterrows(), total=len(grid), desc="Calculating pedestrian path length"):
            intersecting = datasets['hamilton_pedestrian_osm'][datasets['hamilton_pedestrian_osm'].intersects(cell.geometry)]
            if not intersecting.empty:
                grid.at[idx, 'pedestrian_path_length'] = intersecting.to_crs("EPSG:32617").length.sum()

    if 'HSR_Bus_Stops' in datasets and not datasets['HSR_Bus_Stops'].empty:
        for idx, cell in tqdm(grid.iterrows(), total=len(grid), desc="Counting bus stops"):
            grid.at[idx, 'bus_stop_count'] = datasets['HSR_Bus_Stops'].intersects(cell.geometry).sum()

    if 'Traffic_Collisions' in datasets and not datasets['Traffic_Collisions'].empty:
        for idx, cell in tqdm(grid.iterrows(), total=len(grid), desc="Counting traffic collisions"):
            grid.at[idx, 'collision_count'] = datasets['Traffic_Collisions'].intersects(cell.geometry).sum()

    amenity_datasets = ['Hospitals', 'Educational_Institutions', 'Municipal_Services_Centres',
                       'Places_of_Worship', 'Arenas', 'Libraries', 'Museums_and_Galleries',
                       'Recreation_and_Community_Centres', 'Park_Sports_Fields']
    amenity_counts = {cell_id: defaultdict(int) for cell_id in grid['cell_id']}

    for amenity_type in amenity_datasets:
        if amenity_type in datasets and not datasets[amenity_type].empty:
            for idx, cell in tqdm(grid.iterrows(), total=len(grid), desc=f"Processing {amenity_type}"):
                count = datasets[amenity_type].intersects(cell.geometry).sum()
                grid.at[idx, 'amenity_count'] += count
                if count > 0:
                    amenity_counts[cell.cell_id][amenity_type] += count

    for idx, cell in grid.iterrows():
        grid.at[idx, 'amenity_variety'] = len(amenity_counts[cell.cell_id])

    if 'Street_Centreline' in datasets and not datasets['Street_Centreline'].empty:
        streets = datasets['Street_Centreline']
        for idx, cell in tqdm(grid.iterrows(), total=len(grid), desc="Calculating street density"):
            intersecting = streets[streets.intersects(cell.geometry)]
            if not intersecting.empty:
                grid.at[idx, 'street_density'] = intersecting.to_crs("EPSG:32617").length.sum()
        for idx, cell in tqdm(grid.iterrows(), total=len(grid), desc="Estimating intersection density"):
            intersecting = streets[streets.intersects(cell.geometry)]
            if not intersecting.empty:
                grid.at[idx, 'intersection_density'] = len(intersecting) / 2

    return grid

def calculate_bikeability_indicators(grid, datasets):
    grid['bike_lane_length'] = 0.0
    grid['bike_parking_count'] = 0
    grid['bike_share_hubs'] = 0
    grid['bike_friendly_roads'] = 0.0
    grid['bikeway_length'] = 0.0
    grid['bike_collision_count'] = 0
    # Removed bike_trail_length as per request

    if 'hamilton_bike_lanes_osm' in datasets and not datasets['hamilton_bike_lanes_osm'].empty:
        for idx, cell in tqdm(grid.iterrows(), total=len(grid), desc="Calculating bike lane length"):
            intersecting = datasets['hamilton_bike_lanes_osm'][datasets['hamilton_bike_lanes_osm'].intersects(cell.geometry)]
            if not intersecting.empty:
                grid.at[idx, 'bike_lane_length'] = intersecting.to_crs("EPSG:32617").length.sum()

    if 'Bike_Parking' in datasets and not datasets['Bike_Parking'].empty:
        for idx, cell in tqdm(grid.iterrows(), total=len(grid), desc="Counting bike parking"):
            grid.at[idx, 'bike_parking_count'] = datasets['Bike_Parking'].intersects(cell.geometry).sum()

    if 'Hamilton_Bike_Share_Incorporated_Hubs' in datasets and not datasets['Hamilton_Bike_Share_Incorporated_Hubs'].empty:
        for idx, cell in tqdm(grid.iterrows(), total=len(grid), desc="Counting bike share hubs"):
            grid.at[idx, 'bike_share_hubs'] = datasets['Hamilton_Bike_Share_Incorporated_Hubs'].intersects(cell.geometry).sum()

    if 'hamilton_streets_osm' in datasets and not datasets['hamilton_streets_osm'].empty:
        for idx, cell in tqdm(grid.iterrows(), total=len(grid), desc="Calculating bike-friendly roads"):
            if 'bike_friendly' in datasets['hamilton_streets_osm'].columns:
                bike_friendly = datasets['hamilton_streets_osm'][datasets['hamilton_streets_osm']['bike_friendly'] == True]
                intersecting = bike_friendly[bike_friendly.intersects(cell.geometry)]
                if not intersecting.empty:
                    grid.at[idx, 'bike_friendly_roads'] = intersecting.to_crs("EPSG:32617").length.sum()

    if 'Bikeways' in datasets and not datasets['Bikeways'].empty:
        for idx, cell in tqdm(grid.iterrows(), total=len(grid), desc="Calculating bikeway length"):
            intersecting = datasets['Bikeways'][datasets['Bikeways'].intersects(cell.geometry)]
            if not intersecting.empty:
                grid.at[idx, 'bikeway_length'] = intersecting.to_crs("EPSG:32617").length.sum()

    if 'Traffic_Collisions' in datasets and not datasets['Traffic_Collisions'].empty:
        for idx, cell in tqdm(grid.iterrows(), total=len(grid), desc="Counting bike collisions"):
            if 'collision_type' in datasets['Traffic_Collisions'].columns:
                bike_collisions = datasets['Traffic_Collisions'][datasets['Traffic_Collisions']['collision_type'].str.contains('bike', case=False, na=False)]
                grid.at[idx, 'bike_collision_count'] = bike_collisions.intersects(cell.geometry).sum()

    return grid

print("Calculating walkability indicators...")
grid = calculate_walkability_indicators(grid, datasets)

print("Calculating bikeability indicators...")
grid = calculate_bikeability_indicators(grid, datasets)

def calculate_walkability_score(grid):
    if grid['collision_count'].max() > 0:
        grid['safety_score'] = 1 - (grid['collision_count'] / grid['collision_count'].max())
    else:
        grid['safety_score'] = 1

    indicators = ['sidewalk_length', 'street_light_count', 'pedestrian_path_length',
                  'bus_stop_count', 'amenity_count', 'amenity_variety',
                  'street_density', 'intersection_density']
    for indicator in indicators:
        grid[f'norm_{indicator}'] = np.where(grid[indicator].max() > 0, grid[indicator] / grid[indicator].max(), 0)

    weights = {
        'norm_sidewalk_length': 0.15,
        'norm_street_light_count': 0.1,
        'norm_pedestrian_path_length': 0.15,
        'norm_bus_stop_count': 0.1,
        'norm_amenity_count': 0.15,
        'norm_amenity_variety': 0.1,
        'norm_street_density': 0.05,
        'norm_intersection_density': 0.1,
        'safety_score': 0.1
    }
    grid['walkability_score'] = sum(grid[ind] * w for ind, w in weights.items()) * 100
    return grid, weights

def calculate_bikeability_score(grid):
    if grid['bike_collision_count'].max() > 0:
        grid['bike_safety_score'] = 1 - (grid['bike_collision_count'] / grid['bike_collision_count'].max())
    else:
        grid['bike_safety_score'] = 1

    indicators = ['bike_lane_length', 'bike_parking_count', 'bike_share_hubs',
                  'bike_friendly_roads', 'bikeway_length']  # Removed bike_trail_length
    for indicator in indicators:
        grid[f'norm_{indicator}'] = np.where(grid[indicator].max() > 0, grid[indicator] / grid[indicator].max(), 0)

    weights = {
        'norm_bike_lane_length': 0.25,  # Adjusted from 20%
        'norm_bike_parking_count': 0.15,  # Adjusted from 10%
        'norm_bike_share_hubs': 0.10,  # Adjusted from 10%
        'norm_bike_friendly_roads': 0.15,  # Adjusted from 15%
        'norm_bikeway_length': 0.20,  # Adjusted from 20%
        'bike_safety_score': 0.15  # Adjusted from 15%
    }
    grid['bikeability_score'] = sum(grid[ind] * w for ind, w in weights.items()) * 100
    grid['combined_score'] = (grid['walkability_score'] + grid['bikeability_score']) / 2
    return grid, weights

print("Calculating walkability scores...")
grid, walk_weights = calculate_walkability_score(grid)

print("Calculating bikeability scores...")
grid, bike_weights = calculate_bikeability_score(grid)

# Aggregate scores to ward level (overall and within transit service areas)
print("Aggregating scores to ward level...")
# Overall ward scores
grid_with_wards = gpd.sjoin(grid, datasets['Ward_Boundaries'][['geometry', 'WARD']], how='left', predicate='intersects')
grid_with_wards = grid_with_wards.drop(columns=['index_right']).dropna(subset=['WARD'])

# Calculate mean scores per ward (overall)
ward_stats_overall = grid_with_wards.groupby('WARD').agg({
    'walkability_score': 'mean',
    'bikeability_score': 'mean'
}).reset_index()
ward_stats_overall['WARD'] = ward_stats_overall['WARD'].astype(str)
ward_stats_overall['walkability_score'] = ward_stats_overall['walkability_score'].round(2)
ward_stats_overall['bikeability_score'] = ward_stats_overall['bikeability_score'].round(2)

# Scores within transit service areas
if 'Transit_Service_Areas' in datasets and not datasets['Transit_Service_Areas'].empty:
    # Identify grid cells within transit service areas
    grid_within_transit = gpd.sjoin(grid, datasets['Transit_Service_Areas'][['geometry']], how='inner', predicate='intersects')
    grid_within_transit = grid_within_transit.drop(columns=['index_right'])
    # Join with wards to get ward information
    grid_within_transit = gpd.sjoin(grid_within_transit, datasets['Ward_Boundaries'][['geometry', 'WARD']], how='left', predicate='intersects')
    grid_within_transit = grid_within_transit.drop(columns=['index_right']).dropna(subset=['WARD'])
    # Calculate mean scores per ward within transit areas
    ward_stats_transit = grid_within_transit.groupby('WARD').agg({
        'walkability_score': 'mean',
        'bikeability_score': 'mean'
    }).reset_index()
    ward_stats_transit['WARD'] = ward_stats_transit['WARD'].astype(str)
    ward_stats_transit['walkability_score'] = ward_stats_transit['walkability_score'].round(2)
    ward_stats_transit['bikeability_score'] = ward_stats_transit['bikeability_score'].round(2)
else:
    print("Transit_Service_Areas dataset is missing or empty. Transit area scores will be set to 0.")
    ward_stats_transit = pd.DataFrame({'WARD': [str(i) for i in range(1, 16)], 'walkability_score': 0, 'bikeability_score': 0})

# Ensure all wards (1 to 15) are present
all_wards = pd.DataFrame({'WARD': [str(i) for i in range(1, 16)]})
ward_stats_overall = all_wards.merge(ward_stats_overall, on='WARD', how='left').fillna(0)
ward_stats_transit = all_wards.merge(ward_stats_transit, on='WARD', how='left').fillna(0)

# Combine overall and transit scores into one table
ward_stats = ward_stats_overall.merge(ward_stats_transit, on='WARD', suffixes=('_overall', '_transit'))
ward_stats = ward_stats.sort_values('WARD')

# Generate HTML table for ward statistics
table_rows = ""
for idx, row in ward_stats.iterrows():
    ward = row['WARD']
    table_rows += f"""
        <tr>
            <td>{ward}</td>
            <td>{row['walkability_score_overall']}</td>
            <td>{row['bikeability_score_overall']}</td>
            <td>{row['walkability_score_transit']}</td>
            <td>{row['bikeability_score_transit']}</td>
        </tr>
    """

# Generate metrics description with updated bikeability metrics
walk_metrics_html = "<h4>Walkability Metrics</h4><p>The walkability score is calculated based on the following metrics:</p><ul>"
for metric, weight in walk_weights.items():
    metric_name = metric.replace('norm_', '').replace('_', ' ').title()
    data_source = {
        'sidewalk_length': 'Road_Sidewalk',
        'street_light_count': 'Street_Light_Poles_and_Luminaires',
        'pedestrian_path_length': 'hamilton_pedestrian_osm',
        'bus_stop_count': 'HSR_Bus_Stops',
        'amenity_count': 'Hospitals, Educational_Institutions, etc.',
        'amenity_variety': 'Hospitals, Educational_Institutions, etc.',
        'street_density': 'Street_Centreline',
        'intersection_density': 'Street_Centreline',
        'safety_score': 'Traffic_Collisions'
    }.get(metric.replace('norm_', ''), 'N/A')
    walk_metrics_html += f"<li>{metric_name} ({weight*100}%): Data from {data_source}</li>"
walk_metrics_html += "</ul>"

bike_metrics_html = "<h4>Bikeability Metrics</h4><p>The bikeability score is calculated based on the following metrics:</p><ul>"
for metric, weight in bike_weights.items():
    metric_name = metric.replace('norm_', '').replace('_', ' ').title()
    data_source = {
        'bike_lane_length': 'hamilton_bike_lanes_osm',
        'bike_parking_count': 'Bike_Parking',
        'bike_share_hubs': 'Hamilton_Bike_Share_Incorporated_Hubs',
        'bike_friendly_roads': 'hamilton_streets_osm',
        'bikeway_length': 'Bikeways',
        'bike_safety_score': 'Traffic_Collisions'
    }.get(metric.replace('norm_', ''), 'N/A')
    bike_metrics_html += f"<li>{metric_name} ({weight*100}%): Data from {data_source}</li>"
bike_metrics_html += "</ul>"

# Serialize the grid and ward boundaries to GeoJSON
grid_geojson = to_safe_geojson(grid, properties=['cell_id', 'walkability_score', 'bikeability_score', 'combined_score',
                                                 'sidewalk_length', 'street_light_count', 'bus_stop_count', 'amenity_count',
                                                 'bike_lane_length', 'bike_parking_count', 'bike_share_hubs'])
grid_geojson_str = json.dumps(grid_geojson)

wards_geojson = to_safe_geojson(datasets['Ward_Boundaries'], properties=['WARD'])
wards_geojson_str = json.dumps(wards_geojson)

# Generate heat map data
heat_data_walk = [[float(row.geometry.centroid.y), float(row.geometry.centroid.x), float(row.walkability_score)/10]
                  for idx, row in grid.iterrows() if not pd.isna(row.walkability_score)]
heat_data_bike = [[float(row.geometry.centroid.y), float(row.geometry.centroid.x), float(row.bikeability_score)/10]
                  for idx, row in grid.iterrows() if not pd.isna(row.bikeability_score)]
heat_data_walk_str = json.dumps(heat_data_walk)
heat_data_bike_str = json.dumps(heat_data_bike)

# Create the HTML file with Leaflet
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Hamilton Walkability & Bikeability Index</title>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <!-- Google Fonts -->
    <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;500;600&display=swap" rel="stylesheet">
    <!-- Leaflet CSS -->
    <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.4/dist/leaflet.css" />
    <!-- Leaflet JavaScript -->
    <script src="https://unpkg.com/leaflet@1.9.4/dist/leaflet.js"></script>
    <!-- Leaflet Heatmap Plugin -->
    <script src="https://unpkg.com/leaflet.heat@0.2.0/dist/leaflet-heat.js"></script>
    <!-- Leaflet Fullscreen Plugin -->
    <link rel="stylesheet" href="https://api.mapbox.com/mapbox.js/plugins/leaflet-fullscreen/v1.0.1/Leaflet.fullscreen.css" />
    <script src="https://api.mapbox.com/mapbox.js/plugins/leaflet-fullscreen/v1.0.1/Leaflet.fullscreen.min.js"></script>
    <!-- Leaflet Draw Plugin -->
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/leaflet.draw/1.0.4/leaflet.draw.css" />
    <script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet.draw/1.0.4/leaflet.draw.js"></script>
    <!-- Leaflet MousePosition Plugin -->
    <script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet-mouse-position/0.0.1/leaflet.mouseposition.js"></script>
    <style>
        body {{
            margin: 0;
            padding: 0;
            font-family: 'Poppins', sans-serif;
            background-color: #f4f4f9;
        }}
        #map {{
            width: 100%;
            height: 100vh;
        }}
        .heading {{
            position: fixed;
            top: 20px;
            left: 50px;
            background: linear-gradient(135deg, #ffecd2 0%, #fcb69f 100%);
            padding: 15px 25px;
            border-radius: 10px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
            z-index: 1000;
            color: #333;
            text-align: center;
        }}
        .heading h2 {{
            margin: 0;
            font-size: 24px;
            color: #d9534f;
            font-weight: 600;
        }}
        .heading p {{
            margin: 5px 0 0;
            font-size: 14px;
            color: #555;
            font-weight: 400;
        }}
        .legend {{
            background: rgba(255, 255, 255, 0.9);
            padding: 15px;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            font-size: 12px;
            line-height: 1.5;
        }}
        .legend p {{
            text-align: center;
            margin-bottom: 5px;
            font-weight: bold;
            color: #333;
        }}
        .legend div {{
            display: flex;
            align-items: center;
            margin-bottom: 5px;
        }}
        .legend div div {{
            width: 20px;
            height: 20px;
            margin-right: 5px;
            border-radius: 3px;
        }}
        .metrics-table-container {{
            position: fixed;
            top: 150px;
            left: 50px;
            background: rgba(255, 255, 255, 0.95);
            padding: 15px;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
            z-index: 1000;
            max-height: 50vh;
            overflow-y: auto;
            width: 600px;
        }}
        .metrics-table-container h3 {{
            margin: 0 0 10px;
            font-size: 16px;
            color: #d9534f;
            text-align: center;
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
            font-size: 12px;
            color: #333;
        }}
        th, td {{
            padding: 8px;
            text-align: center;
            border-bottom: 1px solid #ddd;
        }}
        th {{
            background-color: #f8f9fa;
            font-weight: bold;
            color: #555;
        }}
        tr:nth-child(even) {{
            background-color: #f9f9f9;
        }}
        tr:hover {{
            background-color: #f1f1f1;
            cursor: pointer;
        }}
        .metrics-details {{
            position: fixed;
            bottom: 50px; /* Moved to bottom-right */
            right: 50px;
            background: rgba(255, 255, 255, 0.95);
            padding: 15px;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
            z-index: 1000;
            width: 300px;
            font-size: 12px;
            color: #333;
        }}
        .metrics-details h4 {{
            margin: 0 0 10px;
            font-size: 16px;
            color: #d9534f;
            text-align: center;
        }}
        .metrics-details p {{
            margin: 5px 0;
        }}
        .metrics-details ul {{
            padding-left: 20px;
            margin: 0;
        }}
        .metrics-details li {{
            margin-bottom: 5px;
        }}
        .leaflet-tooltip {{
            background-color: rgba(255, 255, 255, 0.95);
            border: none;
            border-radius: 5px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2);
            font-size: 12px;
            padding: 8px;
            color: #333;
        }}
        .leaflet-control-layers {{
            background-color: rgba(255, 255, 255, 0.95);
            border-radius: 5px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2);
        }}
    </style>
</head>
<body>
    <div id="map"></div>
    <div class="heading">
        <h2>Hamilton Infrastructure Analysis</h2>
        <p>Promoting Walking and Cycling as Alternative Transport in Hamilton, Ontario</p>
        <p>Estimated Population 2025: 601,268 | Forecasted Population 2030: 645,200</p>
    </div>
    <div class="metrics-table-container">
        <h3>Ward-Level Walkability and Bikeability Scores</h3>
        <table>
            <tr>
                <th>Ward</th>
                <th>Overall Walkability Score</th>
                <th>Overall Bikeability Score</th>
                <th>Transit Walkability Score</th>
                <th>Transit Bikeability Score</th>
            </tr>
            {table_rows}
        </table>
    </div>
    <div class="metrics-details">
        <h4>Metrics for Scores</h4>
        {walk_metrics_html}
        {bike_metrics_html}
    </div>
    <script>
        // Initialize the map
        var map = L.map('map', {{
            center: [43.25, -79.85],
            zoom: 12,
            layers: [],
            fullscreenControl: true
        }});

        // Add base layers
        var positron = L.tileLayer('https://cartodb-basemaps-{{s}}.global.ssl.fastly.net/light_all/{{z}}/{{x}}/{{y}}.png', {{
            attribution: '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> © <a href="http://cartodb.com/attributions">CartoDB</a>',
            subdomains: 'abcd',
            maxZoom: 19
        }}).addTo(map);

        var darkMatter = L.tileLayer('https://cartodb-basemaps-{{s}}.global.ssl.fastly.net/dark_all/{{z}}/{{x}}/{{y}}.png', {{
            attribution: '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> © <a href="http://cartodb.com/attributions">CartoDB</a>',
            subdomains: 'abcd',
            maxZoom: 19
        }});

        var osm = L.tileLayer('https://{{s}}.tile.openstreetmap.org/{{z}}/{{x}}/{{y}}.png', {{
            attribution: '© <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',
            maxZoom: 19
        }});

        var stamenTerrain = L.tileLayer('https://stamen-tiles-{{s}}.a.ssl.fastly.net/terrain/{{z}}/{{x}}/{{y}}.jpg', {{
            attribution: 'Map tiles by <a href="http://stamen.com">Stamen Design</a>, under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>. Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, under <a href="http://www.openstreetmap.org/copyright">ODbL</a>.',
            subdomains: 'abcd',
            maxZoom: 18
        }});

        var stamenWatercolor = L.tileLayer('https://stamen-tiles-{{s}}.a.ssl.fastly.net/watercolor/{{z}}/{{x}}/{{y}}.jpg', {{
            attribution: 'Map tiles by <a href="http://stamen.com">Stamen Design</a>, under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>. Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, under <a href="http://creativecommons.org/licenses/by-sa/3.0">CC BY SA</a>.',
            subdomains: 'abcd',
            maxZoom: 18
        }});

        // Define color functions
        function getWalkColor(score) {{
            score = parseFloat(score);
            if (score >= 80) return '#006400';
            if (score >= 60) return '#32CD32';
            if (score >= 40) return '#FFD700';
            if (score >= 20) return '#FFA500';
            return '#FF0000';
        }}

        function getBikeColor(score) {{
            score = parseFloat(score);
            if (score >= 80) return '#000080';
            if (score >= 60) return '#0000FF';
            if (score >= 40) return '#1E90FF';
            if (score >= 20) return '#87CEEB';
            return '#ADD8E6';
        }}

        function getCombinedColor(score) {{
            score = parseFloat(score);
            if (score >= 80) return '#8B008B';
            if (score >= 60) return '#9932CC';
            if (score >= 40) return '#BA55D3';
            if (score >= 20) return '#DA70D6';
            return '#EE82EE';
        }}

        // Load GeoJSON data
        var gridData = {grid_geojson_str};
        var wardsData = {wards_geojson_str};
        var heatDataWalk = {heat_data_walk_str};
        var heatDataBike = {heat_data_bike_str};

        // Create layers
        var walkLayer = L.geoJSON(gridData, {{
            style: function(feature) {{
                return {{
                    fillColor: getWalkColor(feature.properties.walkability_score),
                    color: 'black',
                    weight: 1,
                    fillOpacity: 0.7
                }};
            }},
            onEachFeature: function(feature, layer) {{
                var props = feature.properties;
                layer.bindTooltip(
                    "Cell ID: " + props.cell_id + "<br>" +
                    "Walkability Score: " + props.walkability_score + "<br>" +
                    "Sidewalk Length: " + props.sidewalk_length + "<br>" +
                    "Street Lights: " + props.street_light_count + "<br>" +
                    "Bus Stops: " + props.bus_stop_count + "<br>" +
                    "Amenities: " + props.amenity_count
                );
            }}
        }});

        var bikeLayer = L.geoJSON(gridData, {{
            style: function(feature) {{
                return {{
                    fillColor: getBikeColor(feature.properties.bikeability_score),
                    color: 'black',
                    weight: 1,
                    fillOpacity: 0.7
                }};
            }},
            onEachFeature: function(feature, layer) {{
                var props = feature.properties;
                layer.bindTooltip(
                    "Cell ID: " + props.cell_id + "<br>" +
                    "Bikeability Score: " + props.bikeability_score + "<br>" +
                    "Bike Lane Length: " + props.bike_lane_length + "<br>" +
                    "Bike Parking: " + props.bike_parking_count + "<br>" +
                    "Bike Share Hubs: " + props.bike_share_hubs
                );
            }}
        }});

        var combinedLayer = L.geoJSON(gridData, {{
            style: function(feature) {{
                return {{
                    fillColor: getCombinedColor(feature.properties.combined_score),
                    color: 'black',
                    weight: 1,
                    fillOpacity: 0.7
                }};
            }},
            onEachFeature: function(feature, layer) {{
                var props = feature.properties;
                layer.bindTooltip(
                    "Cell ID: " + props.cell_id + "<br>" +
                    "Combined Score: " + props.combined_score + "<br>" +
                    "Walkability Score: " + props.walkability_score + "<br>" +
                    "Bikeability Score: " + props.bikeability_score
                );
            }}
        }});

        var walkHeatLayer = L.heatLayer(heatDataWalk, {{
            radius: 15,
            blur: 10,
            maxZoom: 18,
            gradient: {{0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 0.8: 'orange', 1: 'red'}}
        }});

        var bikeHeatLayer = L.heatLayer(heatDataBike, {{
            radius: 15,
            blur: 10,
            maxZoom: 18,
            gradient: {{0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 0.8: 'orange', 1: 'red'}}
        }});

        var wardsLayer = L.geoJSON(wardsData, {{
            style: function(feature) {{
                return {{
                    fillColor: 'transparent',
                    color: 'black',
                    weight: 2,
                    dashArray: '5,5'
                }};
            }},
            onEachFeature: function(feature, layer) {{
                var wardName = feature.properties.WARD || 'Unknown Ward';
                layer.bindTooltip("Ward " + wardName);
            }}
        }});

        // Layer control
        var baseMaps = {{
            "CartoDB Positron": positron,
            "CartoDB Dark Matter": darkMatter,
            "OpenStreetMap": osm,
            "Stamen Terrain": stamenTerrain,
            "Stamen Watercolor": stamenWatercolor
        }};

        var overlayMaps = {{
            "Walkability Score": walkLayer,
            "Bikeability Score": bikeLayer,
            "Combined Score": combinedLayer,
            "Walkability Heat": walkHeatLayer,
            "Bikeability Heat": bikeHeatLayer,
            "Ward Boundaries": wardsLayer
        }};

        L.control.layers(baseMaps, overlayMaps, {{collapsed: false}}).addTo(map);

        // Add default layers
        walkLayer.addTo(map);
        wardsLayer.addTo(map);

        // Add Draw control
        var drawnItems = new L.FeatureGroup();
        map.addLayer(drawnItems);
        var drawControl = new L.Control.Draw({{
            edit: {{ featureGroup: drawnItems }}
        }});
        map.addControl(drawControl);
        map.on('draw:created', function(e) {{
            var layer = e.layer;
            drawnItems.addLayer(layer);
        }});

        // Add MousePosition control
        L.control.mousePosition({{
            position: 'bottomleft',
            separator: ' | ',
            lngFirst: true,
            numDigits: 5,
            prefix: 'Coordinates: '
        }}).addTo(map);
    </script>
</body>
</html>
"""

# Save the HTML file
output_file = "hamilton_walkability_bikeability_index.html"
with open(output_file, 'w') as f:
    f.write(html_content)
print(f"Map saved to {output_file}")

# Display the map inline (for Google Colab or Jupyter)
from IPython.display import IFrame
IFrame(src=output_file, width=1000, height=600)

Loading datasets...
Loaded Street_Light_Poles_and_Luminaires with 21276 features


Loaded Traffic_Collisions with 82981 features
Loaded hamilton_bike_lanes_osm with 774 features


Loaded hamilton_pedestrian_osm with 8001 features
Loaded hamilton_streets_osm with 22866 features
Loaded Road_Sidewalk with 44444 features
Loaded Street_Centreline with 19837 features
Loaded Transit_Service_Areas with 1 features
Loaded HSR_Bus_Stops with 2371 features
Loaded Hospitals with 9 features
Loaded EMS_Stations with 19 features
Loaded Police_Stations with 4 features
Loaded Educational_Institutions with 227 features
Loaded Municipal_Services_Centres with 6 features
Loaded Places_of_Worship with 356 features
Loaded Arenas with 25 features
Loaded Spray_Pads with 68 features
Loaded Libraries with 23 features
Loaded Museums_and_Galleries with 12 features
Loaded Public_Art_and_Monuments with 51 features
Loaded City_Waterfalls with 103 features
Loaded Recreation_and_Community_Centres with 75 features
Loaded Park_Sports_Fields with 713 features
Loaded Campgrounds with 12 features
Loaded Ward_Boundaries with 15 features
Loaded Bikeways with 3657 features
Loaded Bike_Parking with 822 fe

Calculating sidewalk length:   0%|          | 0/7904 [00:00<?, ?it/s]

Counting street lights:   0%|          | 0/7904 [00:00<?, ?it/s]

...

Counting bike share hubs:   0%|          | 0/7904 [00:00<?, ?it/s]

Calculating bike-friendly roads:   0%|          | 0/7904 [00:00<?, ?it/s]

Calculating bikeway length:   0%|          | 0/7904 [00:00<?, ?it/s]

Counting bike collisions:   0%|          | 0/7904 [00:00<?, ?it/s]

Calculating walkability scores...
Calculating bikeability scores...
Aggregating scores to ward level...
Map saved to hamilton_walkability_bikeability_index.html
